## VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Import .csv from WeatherPy (Part 1) into a DataFrame

In [2]:
source_file = './output_data/output_cities_AR.csv'
cities_df = pd.read_csv(source_file, sep = ",", low_memory=False)
cities_df['API Call Date'] = pd.to_datetime(cities_df['API Call Date'], unit='s')
cities_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,API Call Date
0,Tasiilaq,65.61,-37.64,33.80,91,100,5.82,GL,2020-10-06 22:04:22
1,Ushuaia,-54.80,-68.30,37.40,76,40,24.16,AR,2020-10-06 22:02:36
2,Khanbari,27.37,87.20,68.18,66,22,2.48,NP,2020-10-06 22:04:23
3,Toora-Khem,52.47,96.11,23.74,95,78,1.74,RU,2020-10-06 22:04:23
4,San Martín,-33.08,-68.47,78.80,21,0,8.05,AR,2020-10-06 22:04:23


In [3]:
n_rows, n_cols = cities_df.shape
print(f'There are {n_rows} rows and {n_cols} columns.')

There are 561 rows and 9 columns.


In [4]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   City           561 non-null    object        
 1   Latitude       561 non-null    float64       
 2   Longitude      561 non-null    float64       
 3   Max Temp       561 non-null    float64       
 4   Humidity       561 non-null    int64         
 5   Cloudiness     561 non-null    int64         
 6   Wind Speed     561 non-null    float64       
 7   Country        558 non-null    object        
 8   API Call Date  561 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 39.6+ KB


### Humidity Heat Map

In [5]:
# Configure gmaps w API key
gmaps.configure(api_key = g_key)
locations = cities_df[["Latitude", "Longitude"]]
weights = cities_df['Humidity']

In [6]:
# Use the Lat and Lng as locations and Humidity as the weight.
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Add Heatmap layer to map.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,
                                 weights = weights, dissipating = False,
                                 max_intensity = 300, point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create a new Data Frame
Narrow list to include only the cities that meet ALL of the following criteria:
* Max Temperature between 70 - 80 degrees. 
* Wind Speed less than 10 mph.
* Zero Cloudiness 

In [8]:
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & 
                         (cities_df['Max Temp'] > 70) 
                              & (cities_df['Wind Speed'] < 10) & (cities_df['Humidity'] <=50)].copy() # I like clouds

### Hotel Map

In [9]:
# * Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop=True).head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,API Call Date,Hotel Name
0,San Martín,-33.08,-68.47,78.80,21,0,8.05,AR,2020-10-06 22:04:23,
1,Castro,-24.79,-50.01,75.88,41,49,2.98,BR,2020-10-06 22:04:42,
2,Kiryat Gat,31.61,34.76,75.99,43,0,1.99,IL,2020-10-06 22:05:02,
3,Barentu,15.11,37.59,77.41,39,0,2.46,ER,2020-10-06 22:05:13,
4,Sojat,25.92,73.67,77.14,39,100,5.03,IN,2020-10-06 22:05:24,


In [10]:
# Set parameters to search for hotels with 5000 meters.
radius = 5000

# Use Google Places API for each city's coordinates.
base_url_nearby = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Store the first Hotel result into the DataFrame.
params = {
    "types": "lodging",
    "radius": radius,
    "key": g_key}

for index,row in hotel_df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
# RUN REQUEST
    response = requests.get(base_url_nearby, params)
# TEST RESULTS
    results = response.json()
# PRINT RESULTS
    try:
     # record the hotel name
        hotel_df['Hotel Name'][index] = results["results"][0]["name"]
    except IndexError:
        print(f"No hotels available near {hotel_df['City'][index]}.\n")
        hotel_df['Hotel Name'][index] = "none available"
hotel_df

No hotels available near Barentu.



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,API Call Date,Hotel Name
4,San Martín,-33.08,-68.47,78.80,21,0,8.05,AR,2020-10-06 22:04:23,Finca Vivencias
74,Castro,-24.79,-50.01,75.88,41,49,2.98,BR,2020-10-06 22:04:42,CHACARA BAILLY
153,Kiryat Gat,31.61,34.76,75.99,43,0,1.99,IL,2020-10-06 22:05:02,OlusHome
191,Barentu,15.11,37.59,77.41,39,0,2.46,ER,2020-10-06 22:05:13,none available
239,Sojat,25.92,73.67,77.14,39,100,5.03,IN,2020-10-06 22:05:24,AASHIRWAD BHAWAN
289,Oodweyne,9.41,45.06,76.51,28,24,4.72,SO,2020-10-06 22:05:37,Eng Hiirey House
315,Najrān,17.49,44.13,71.60,21,0,2.10,SA,2020-10-06 22:05:45,محمد هادي ال فطيح
331,Rundu,-17.93,19.77,76.37,22,20,4.52,NaN,2020-10-06 22:05:49,Millennia Estate
378,Jalpa,21.63,-102.97,78.80,9,75,9.17,MX,2020-10-06 22:06:02,"Estacionamiento ""El Nogal"""
406,Anagé,-14.61,-41.14,79.52,49,2,6.35,BR,2020-10-06 22:06:10,Lagoa Torta Anage Bahia


In [11]:
# Using the template provided, add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Plot markers on top of the heatmap.
new_map = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(new_map)
fig

Figure(layout=FigureLayout(height='420px'))